# Problema do Caixeiro Viajante

## INICIO

In [ ]:
!pip install osmnx descartes shapely
!pip install folium

In [ ]:
!pip install acopy


In [ ]:
import osmnx as ox
import networkx as nx
import folium
import geopandas as gpd
import pandas as pd

In [ ]:
G = gpd.read_file('/content/geojs-35-mun.json')

In [ ]:
G['centroid'] = G['geometry'].centroid

<ipython-input-3-4dc613a3ecd5>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  G['centroid'] = G['geometry'].centroid


In [ ]:
G

,id,name,description,geometry,centroid
0,3500105,Adamantina,Adamantina,"POLYGON ((-51.05787 -21.39888, -51.05365 -21.4...",POINT (-51.05616 -21.57630)
1,3500204,Adolfo,Adolfo,"POLYGON ((-49.65478 -21.20607, -49.63847 -21.2...",POINT (-49.65281 -21.28808)
2,3500303,Aguaí,Aguaí,"POLYGON ((-47.20890 -21.97129, -47.20297 -21.9...",POINT (-47.03971 -22.04960)
3,3500402,Águas da Prata,Águas da Prata,"POLYGON ((-46.70755 -21.82895, -46.70380 -21.8...",POINT (-46.69131 -21.91376)
4,3500501,Águas de Lindóia,Águas de Lindóia,"POLYGON ((-46.61147 -22.43496, -46.60302 -22.4...",POINT (-46.60305 -22.47405)
...,...,...,...,...,...
640,3557006,Votorantim,Votorantim,"POLYGON ((-47.37268 -23.51783, -47.37047 -23.5...",POINT (-47.40599 -23.57923)
641,3557105,Votuporanga,Votuporanga,"POLYGON ((-50.02073 -20.33347, -50.01275 -20.3...",POINT (-49.98817 -20.45815)
642,3557154,Zacarias,Zacarias,"POLYGON ((-49.99521 -21.03253, -49.98434 -21.0...",POINT (-50.04907 -21.12492)
643,3557204,Chavantes,Chavantes,"POLYGON ((-49.68539 -22.94866, -49.68396 -22.9...",POINT (-49.72615 -23.04595)


## 1: Melhor otimização de uma rota

In [ ]:
import numpy as np

In [ ]:
import folium
import numpy as np
from geopy.distance import great_circle

# Função para obter coordenadas do centróide
def get_centroid_coords(endereco):
    # Suponha que você tenha uma função `ox.geocode()` para obter as coordenadas
    coords = ox.geocode(endereco)
    return coords[0], coords[1]  # Retorna lat, lon

# Função para calcular a distância entre duas coordenadas
def calcular_distancia(coord1, coord2):
    return great_circle(coord1, coord2).kilometers

# Função para calcular a distância total de uma rota
def calcular_distancia_total(rota, coords):
    distancia_total = 0
    for i in range(len(rota) - 1):
        distancia_total += calcular_distancia(coords[rota[i]], coords[rota[i+1]])
    return distancia_total

# Função para atualizar os feromônios em uma rota
def atualizar_feromonios(feromonios, rota, evap_rate, q):
    for i in range(len(rota) - 1):
        feromonios[rota[i], rota[i+1]] += q / calcular_distancia_total(rota, coords)
    return feromonios

# Lista de endereços
enderecos = [
    'São José dos Campos, SP',
    'Poa, SP',
    'Bariri, SP',
    'Campinas, SP',
    'Boa Esperança do Sul, SP',
    'Rio Claro, SP',
    'Poços de Caldas, SP',
    'São Pedro do Sul, SP',
    'Holambra, SP',
    'São José do Rio Preto, SP',
    'Joanópolis, SP',
    'São João de Meriti, SP',#12
    'São João de Pirapora, SP',
    'São Luiz do Paraitinga, SP',
    'Ribeirão Preto, SP',
    'Marília, SP',
    'Presidente Prudente, SP',
    'Americana, SP',
    'Assis, SP',
    'Rio Claro'#21
]

# Lista para armazenar as coordenadas dos centróides
coords = [get_centroid_coords(end) for end in enderecos]

# Parâmetros do algoritmo de ACO
num_ants = 10
num_iterations = 100
evap_rate = 0.1
q = 100

# Inicializar os feromônios em todas as arestas
feromonios = np.ones((len(coords), len(coords)))

# Executar o algoritmo de ACO
for _ in range(num_iterations):
    for _ in range(num_ants):
        # Inicializar uma formiga aleatória
        rota = np.random.permutation(len(coords))
        # Atualizar os feromônios na rota
        feromonios = atualizar_feromonios(feromonios, rota, evap_rate, q)

# Encontrar a melhor rota com base nos feromônios
melhor_rota = np.argmax(feromonios, axis=1)
melhor_distancia = calcular_distancia_total(melhor_rota, coords)

# Mapa com os marcadores dos centróides e linhas retas entre eles
mapa = folium.Map(location=[coords[0][0], coords[0][1]], zoom_start=10)  # Definindo o centro do mapa com as coordenadas do primeiro centróide

# Adiciona marcadores para os centróides
for j in range(len(enderecos)):
    D = coords[j]
    folium.Marker(location=[D[0], D[1]], tooltip=f'{enderecos[j]}').add_to(mapa)

# Adiciona linhas retas entre os centróides no mapa
for i in range(len(coords)):
    for j in range(i + 1, len(coords)):
        if (i == melhor_rota[j] and j == melhor_rota[i]):
            folium.PolyLine(locations=[(coords[i][0], coords[i][1]), (coords[j][0], coords[j][1])], color='green', dash_array='5').add_to(mapa)
        else:
            folium.PolyLine(locations=[(coords[i][0], coords[i][1]), (coords[j][0], coords[j][1])], color='red', dash_array='5').add_to(mapa)

# Exibe o mapa
mapa


In [ ]:
import acopy

In [ ]:
import folium
import numpy as np
from geopy.distance import great_circle
import acopy
import networkx as nx
import osmnx as ox

# Função para obter coordenadas do centróide
def get_centroid_coords(endereco):
    # Obtém as coordenadas do centróide do endereço fornecido
    coords = ox.geocode(endereco)
    return coords[0], coords[1]  # Retorna lat, lon

# Lista de endereços
enderecos = [
    'São José dos Campos, SP',
    'Poa, SP',
    'Bariri, SP',
    'Campinas, SP',
    'Boa Esperança do Sul, SP',
    'Rio Claro, SP',
    'Poços de Caldas, SP',
    'São Pedro do Sul, SP',
    'Holambra, SP',
    'São José do Rio Preto, SP',
    'Joanópolis, SP',
    'São João de Meriti, SP',
    'Areias, SP',
    'São João de Pirapora, SP',
    'São Luiz do Paraitinga, SP',
    'Ribeirão Preto, SP',
    'Marília, SP',
    'Presidente Prudente, SP',
    'Americana, SP',
    'Assis, SP',
    'Rio Claro'
]

# Lista para armazenar as coordenadas dos centróides
coord = [get_centroid_coords(end) for end in enderecos]

# Criação do grafo para o ACO
G = nx.Graph()

# Adicionando nós ao grafo
for i, (lat, lon) in enumerate(coord):
    G.add_node(i, pos=(lat, lon))

# Adicionando arestas com as distâncias calculadas
for i in range(len(coord)):
    for j in range(i + 1, len(coord)):
        distance = great_circle(coord[i], coord[j]).kilometers
        G.add_edge(i, j, weight=distance)

# Configuração do otimizador de colônia de formigas
ant_colony = acopy.Colony(alpha=1, beta=3)
solver = acopy.Solver(rho=0.1, q=1)

# Definindo o número de iterações
iterations = 1000  # Número de iterações desejado

# Resolução do problema usando ACO
tour = solver.solve(G, ant_colony, limit=iterations)

# Extração do caminho encontrado pela ACO
best_path = tour.nodes

# Mapa com os marcadores dos centróides e linhas da melhor rota encontrada
mapa = folium.Map(location=[coord[0][0], coord[0][1]], zoom_start=10)

# Adiciona marcadores para os centróides
for j in range(len(enderecos)):
    D = coord[j]
    folium.Marker(location=[D[0], D[1]], tooltip=f'{enderecos[j]}').add_to(mapa)

# Adiciona a melhor rota encontrada pela ACO no mapa
for i in range(len(best_path) - 1):
    folium.PolyLine(locations=[(coord[best_path[i]][0], coord[best_path[i]][1]), (coord[best_path[i + 1]][0], coord[best_path[i + 1]][1])], color='blue').add_to(mapa)

# Conecta o último ponto ao primeiro para fechar o circuito
folium.PolyLine(locations=[(coord[best_path[-1]][0], coord[best_path[-1]][1]), (coord[best_path[0]][0], coord[best_path[0]][1])], color='blue').add_to(mapa)

# Exibe o mapa
mapa


In [ ]:
# Função para calcular a distância total do percurso
def calculate_total_distance(path, coord):
    total_distance = 0
    for i in range(len(path) - 1):
        total_distance += great_circle(coord[path[i]], coord[path[i + 1]]).kilometers
    # Adiciona a distância de retorno ao ponto inicial
    total_distance += great_circle(coord[path[-1]], coord[path[0]]).kilometers
    return total_distance

# Calcula a distância total do percurso
total_distance = calculate_total_distance(best_path, coord)
print(f"Distância total do percurso: {total_distance:.2f} km")


Distância total do percurso: 1848.76 km


## 3: passando por todos os municipios


In [ ]:
!pip install py2opt

In [ ]:
import random2
import py2opt

In [ ]:
import geopandas as gpd
import folium
import numpy as np
from geopy.distance import great_circle
from itertools import combinations
from joblib import Parallel, delayed
import time
import random2
from py2opt.solver import Solver

# Classe RouteFinder importada do seu código
class RouteFinder:
    def __init__(self, distance_matrix, cities_names, iterations=5, writer_flag=False, method='py2opt', return_to_begin=False, verbose=True):
        self.distance_matrix = distance_matrix
        self.iterations = iterations
        self.return_to_begin = return_to_begin
        self.writer_flag = writer_flag
        self.cities_names = cities_names
        self.verbose = verbose

    def solve(self):
        start_time = round(time.time() * 1000)
        elapsed_time = 0
        iteration = 0
        best_distance = 0
        best_route = []
        best_distances = []

        while iteration < self.iterations:
            num_cities = len(self.distance_matrix)
            if self.verbose:
                print(round(elapsed_time), 'msec')
            initial_route = [0] + random2.sample(range(1, num_cities), num_cities - 1)
            if self.return_to_begin:
                initial_route.append(0)
            tsp = Solver(self.distance_matrix, initial_route)
            new_route, new_distance, distances = tsp.two_opt()

            if iteration == 0:
                best_distance = new_distance
                best_route = new_route
            else:
                pass

            if new_distance < best_distance:
                best_distance = new_distance
                best_route = new_route
                best_distances = distances

            elapsed_time = round(time.time() * 1000) - start_time
            iteration += 1

        if self.writer_flag:
            self.writer(best_route, best_distance, self.cities_names)

        if self.cities_names:
            best_route = [self.cities_names[i] for i in best_route]
            return best_distance, best_route
        else:
            return best_distance, best_route

    @staticmethod
    def writer(best_route, best_distance, cities_names):
        with open("../results.txt", "w+") as f:
            for i in best_route:
                f.write(cities_names[i])
                f.write("\n")
                print(cities_names[i])
            f.write(str(best_distance))

# Carregando o arquivo JSON com os municípios de São Paulo
G = gpd.read_file('/content/geojs-35-mun.json')

# Função para obter as coordenadas do centróide
def get_centroid_coords(geometry):
    centroid = geometry.centroid
    return centroid.y, centroid.x  # Retorna lat, lon

# Função para verificar se duas linhas se cruzam
def lines_intersect(p1, p2, q1, q2):
    def ccw(a, b, c):
        return (c[1] - a[1]) * (b[0] - a[0]) > (b[1] - a[1]) * (c[0] - a[0])
    return ccw(p1, q1, q2) != ccw(p2, q1, q2) and ccw(p1, p2, q1) != ccw(p1, p2, q2)

# Função para corrigir cruzamentos no caminho
def correct_crossings(path, coords):
    improved = True
    while improved:
        improved = False
        for i in range(len(path) - 1):
            for j in range(i + 2, len(path) - 1):
                if lines_intersect(coords[path[i]], coords[path[i + 1]], coords[path[j]], coords[path[j + 1]]):
                    # Swap the segments to remove the intersection
                    path[i + 1:j + 1] = reversed(path[i + 1:j + 1])
                    improved = True
    return path

# Extraindo os centróides de todos os municípios
coord = [get_centroid_coords(geom) for geom in G.geometry]

# Limitar a quantidade de dados para um teste inicial
coord = coord[:645]

# Criação da matriz de distâncias
print("Calculando a matriz de distâncias...")
start_time = time.time()
num_cities = len(coord)
distance_matrix = np.zeros((num_cities, num_cities))

def calc_distance(i, j):
    return great_circle(coord[i], coord[j]).kilometers

results = Parallel(n_jobs=-1)(delayed(calc_distance)(i, j) for i, j in combinations(range(num_cities), 2))

for (i, j), dist in zip(combinations(range(num_cities), 2), results):
    distance_matrix[i, j] = dist
    distance_matrix[j, i] = dist
print(f"Matriz de distâncias calculada em {time.time() - start_time:.2f} segundos.")

# Nomes das cidades para referência
cities_names = [G.iloc[i]['name'] for i in range(len(coord))]

# Inicializando e resolvendo o problema com RouteFinder
route_finder = RouteFinder(distance_matrix, cities_names, iterations=4000, writer_flag=True, return_to_begin=True)
best_distance, best_route = route_finder.solve()

print(f"Melhor distância: {best_distance}")
print(f"Melhor rota: {best_route}")

# Mapa com os marcadores dos centróides e linhas da melhor rota encontrada
mapa = folium.Map(location=[coord[0][0], coord[0][1]], zoom_start=7)

# Remover a seção que adiciona marcadores para os centróides
# print("Adicionando marcadores ao mapa...")
# for j in range(len(coord)):
#     D = coord[j]
#     municipio_nome = G.iloc[j]['name']  # Substitua 'name' pelo campo apropriado para o nome do município no seu JSON
#     folium.Marker(location=[D[0], D[1]], tooltip=f'{municipio_nome}').add_to(mapa)

# Adiciona a melhor rota encontrada pela ACO no mapa
print("Adicionando a melhor rota ao mapa...")
for i in range(len(best_route) - 1):
    folium.PolyLine(locations=[(coord[cities_names.index(best_route[i])][0], coord[cities_names.index(best_route[i])][1]), (coord[cities_names.index(best_route[i + 1])][0], coord[cities_names.index(best_route[i + 1])][1])], color='blue').add_to(mapa)

# Conecta o último ponto ao primeiro para fechar o circuito
folium.PolyLine(locations=[(coord[cities_names.index(best_route[-1])][0], coord[cities_names.index(best_route[-1])][1]), (coord[cities_names.index(best_route[0])][0], coord[cities_names.index(best_route[0])][1])], color='blue').add_to(mapa)

# Exibe o mapa
print("Exibindo o mapa.")
mapa


Calculando a matriz de distâncias...
Matriz de distâncias calculada em 5.39 segundos.
0 msec
2554 msec
5036 msec
9390 msec
12176 msec
14659 msec
17195 msec
21180 msec
24829 msec
27299 msec
29863 msec
32333 msec
36448 msec
39011 msec
41498 msec
44077 msec
47175 msec
50945 msec
53470 msec
55985 msec
58530 msec
62373 msec
66038 msec
69231 msec
71715 msec
75105 msec
78976 msec
81836 msec
84678 msec
87154 msec
91486 msec
94340 msec
96875 msec
99311 msec
102586 msec
106205 msec
109008 msec
111528 msec
114042 msec
118241 msec
121139 msec
123650 msec
126855 msec
130413 msec
134086 msec
138143 msec
140667 msec
145286 msec
147852 msec
150384 msec
152896 msec
156218 msec
160003 msec
162472 msec
165276 msec
167764 msec
171564 msec
174363 msec
177544 msec
180080 msec
183082 msec
186995 msec
189509 msec
192004 msec
194524 msec
198663 msec
201540 msec
204376 msec
207218 msec
210201 msec
214135 msec
216620 msec
219109 msec
221620 msec
225187 msec
228215 msec
231434 msec
233912 msec
236392 msec
240685 

In [ ]:
# Função para calcular a distância total do percurso
def calculate_total_distance(route, distance_matrix):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance_matrix[route[i], route[i + 1]]
    # Adiciona a distância de retorno ao ponto inicial, se necessário
    if route[0] == route[-1]:
        total_distance += distance_matrix[route[-2], route[-1]]
    return total_distance

# Converte a melhor rota em índices da matriz de distâncias
best_route_indices = [cities_names.index(city) for city in best_route]

# Calcula a distância total do percurso
total_distance = calculate_total_distance(best_route_indices, distance_matrix)

print(f"Distância total do percurso: {total_distance:.2f} km")


Distância total do percurso: 11789.04 km
